In [1]:
import os
from openai import OpenAI
import pandas as pd
from datetime import date

# Working Dir.
os.chdir('/Users/fogellmcmuffin/Documents/ra/team_discussions/AI/')
os.environ['OPENAI_API_KEY'] = 'sk-proj-BBrbzGnsFwklndehaTC1T3BlbkFJEGPQt0QfhWkwp9ePuxaK'

In [2]:
####################
## Model Settings ##
####################

# Calling for OpenAI client
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
    organization='org-WLFAmqjnKmywM0wd6loMyGJq',    # RA_WORK
    project='proj_vOr6WeeCFk5IjZLCdksFLWUd',    # IRPD_CODING
)

# Model Settings
MODEL = 'gpt-4o-2024-05-13'
TEMPERATURE = 0
MAX_TOKENS = 1300
TOP_P = 1
FREQUENCY_PENALTY = 0
PRESENCE_PENALTY = 0

In [3]:
######################
## Prompt Functions ##
######################

def file_to_string(file_path):  # File read-to-string functions
  with open(file_path, 'r') as file:
    k = file.read()
  return str(k)


def write_file(file_path, file_write):  # File write function
  with open(file_path, 'w') as file:
    file.write(file_write)     


def test_info(test, data_name):  # Function to get test info
  info = str(
    'ChatGPT Model Information:' + '\n' +
    'format: OpenAI API' + '\n' +
    'model: ' + str(MODEL) + '\n' +
    'temperature: ' + str(TEMPERATURE) + '\n' +
    'max tokens: ' + str(MAX_TOKENS) + '\n' +
    'top p: ' + str(TOP_P) + '\n' +
    'frequency penalty: ' + str(FREQUENCY_PENALTY) + '\n' +
    'presence penalty: ' + str(PRESENCE_PENALTY) + '\n\n' +
    'Test Information:' + '\n' + 
    'test: ' + test + '\n' +
    'data: ' + data_name + '\n' +
    'date: ' + str(date.today()) + '\n'
  )
  return info


def GPT_response(sys, user):  # Simple GPT response function
  # Requesting chat completion
  response = client.chat.completions.create(
    model=MODEL,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    top_p=TOP_P,
    frequency_penalty=FREQUENCY_PENALTY,
    presence_penalty=PRESENCE_PENALTY,
    messages=[
      {"role": "system", "content": str(sys)},
      {"role": "user", "content": str(user)}
    ]
  )
  output = response.choices[0].message.content  # GPT response var
  
  return output


def test_GPT(sys_path, user_path, df_name):  # Function for non-official tests
  # utf-8 encoding prompts
  sys = file_to_string(sys_path)
  user = file_to_string(user_path)
  
  # Generating new folder for subtest
  dir_numbers = [int(d) for d in os.listdir('output/_subtests/') if d.isdigit()]
  max_number = max(dir_numbers) # Getting max number in subtest directory
  unique_id = max_number + 1
  test_dir = os.path.join('output/_subtests/', str(unique_id))
  os.makedirs(test_dir, exist_ok=False)  # Attempt to create the directory
  info = test_info(f"Sub-test {unique_id}", df_name)  # Getting test info
  info_path = os.path.join(test_dir, f'{unique_id}__subtest_info.txt')
  write_file(info_path, info)
  
  output = GPT_response(sys, user) # Requesting chat completion
  
  # Creating paths for prompts & GPT response
  sys_prmpt_path = os.path.join(test_dir, f'{unique_id}_sys_prmpt.txt')
  user_prmpt_path = os.path.join(test_dir, f'{unique_id}_user_prmpt.txt')
  response_path = os.path.join(test_dir, f'{unique_id}_response.txt')
  
  # Writing .txt files for prompts & GPT response
  write_file(sys_prmpt_path, str(sys))
  write_file(user_prmpt_path, str(user))
  write_file(response_path, str(output))
  
  return print(f'Sub-test {unique_id} complete')


def stage_1_output(treatment, test):  # Stage 1 function: Creating categories
  # utf-8 encoding prompts
  sys_ucoop = file_to_string(f'code/prompts/ucoop/{treatment}/sys_1_{treatment}_ucoop.md')
  sys_udef = file_to_string(f'code/prompts/udef/{treatment}/sys_1_{treatment}_udef.md')
  user_ucoop = file_to_string(f'code/prompts/ucoop/{treatment}/user_1_{treatment}_ucoop.md')
  user_udef = file_to_string(f'code/prompts/udef/{treatment}/user_1_{treatment}_udef.md')
  window_prompts = [['ucoop', sys_ucoop, user_ucoop], ['udef', sys_udef, user_udef]]
  
  # Making test directory
  test_dir = os.path.join('output/', test)
  os.makedirs(test_dir, exist_ok=True)
  
  # Test info
  info = test_info(f"Test {test[5:]}", f'RAsum_{treatment}_ucoop.csv & RAsum_{treatment}_udef.csv')
  info_path = os.path.join(test_dir, f't{test[5:]}_test_info.txt')
  write_file(info_path, info)
  
  # Creating stage 1 windows
  for i in window_prompts:
    stage_dir = os.path.join(test_dir, f'stage_1_{i[0]}')
    os.makedirs(stage_dir, exist_ok=False)  # Stage directory
    
    # GPT request output
    output = GPT_response(i[1], i[2])
    
    # Creating paths for prompts & GPT response
    sys_prmpt_path = os.path.join(stage_dir, f't{test[5:]}_stg_1_{i[0]}_sys_prmpt.txt')
    user_prmpt_path = os.path.join(stage_dir, f't{test[5:]}_stg_1_{i[0]}_user_prmpt.txt')
    response_path = os.path.join(stage_dir, f't{test[5:]}_stg_1_{i[0]}_response.txt')
    
    # Writing .txt files for prompts & GPT response
    write_file(sys_prmpt_path, i[1])
    write_file(user_prmpt_path, i[2])
    write_file(response_path, str(output))
  
  return print("Stage 1 Complete")


def stage_1r_output(treatment, test):  # Stage 1 category refinement function
  # utf-8 encoding prompts
  sys_ucoop = file_to_string(f'code/prompts/ucoop/{treatment}/sys_1r_{treatment}_ucoop.md')
  sys_udef = file_to_string(f'code/prompts/udef/{treatment}/sys_1r_{treatment}_udef.md')
  user_ucoop = file_to_string(f'code/prompts/ucoop/{treatment}/user_1r_{treatment}_ucoop.md')
  user_udef = file_to_string(f'code/prompts/udef/{treatment}/user_1r_{treatment}_udef.md')
  window_prompts = [['ucoop', sys_ucoop, user_ucoop], ['udef', sys_udef, user_udef]]
  
  test_dir = f'output/{test}/'    # Test directory
  
  for i in window_prompts:
    stage_dir = os.path.join(test_dir, f'stage_1r_{i[0]}')
    os.makedirs(stage_dir, exist_ok=False)  # Stage directory
    
    # GPT request output
    output = GPT_response(i[1], i[2])
    
    # Creating paths for prompts & GPT response
    sys_prmpt_path = os.path.join(stage_dir, f't{test[5:]}_stg_1r_{i[0]}_sys_prmpt.txt')
    user_prmpt_path = os.path.join(stage_dir, f't{test[5:]}_stg_1r_{i[0]}_user_prmpt.txt')
    response_path = os.path.join(stage_dir, f't{test[5:]}_stg_1r_{i[0]}_response.txt')
    
    # Writing .txt files for prompts & GPT response
    write_file(sys_prmpt_path, i[1])
    write_file(user_prmpt_path, i[2])
    write_file(response_path, str(output))
  
  return print("Stage 1 Refinement Complete")


def stage_2_output(treatment, test, df_ucoop, df_udef): # Stage 2 function: Assigning categories to individual summaries (recursive)
  # utf-8 encoding system prompt
  sys_ucoop = file_to_string(f'code/prompts/ucoop/{treatment}/sys_2_{treatment}_ucoop.md')
  sys_udef = file_to_string(f'code/prompts/udef/{treatment}/sys_2_{treatment}_udef.md')
  window_prompts = [['ucoop', sys_ucoop, df_ucoop], ['udef', sys_udef, df_udef]]
  
  test_dir = f'output/{test}/'    # Test directory
  
  for i in window_prompts:
    stage_dir = os.path.join(test_dir, f'stage_2_{i[0]}')
    os.makedirs(stage_dir, exist_ok=False)  # Stage directory
    
    # Writing .txt system prompt (the same for every request)
    sys = i[1]    # System prompt for ucoop or udef data
    sys_prmpt_path = os.path.join(stage_dir, f't{test[5:]}_stg_2_{i[0]}_sys_prmpt.txt')
    write_file(sys_prmpt_path, sys)
    
    # Prompt & Response paths
    prompt_path = os.path.join(stage_dir, 'prompts')
    response_path = os.path.join(stage_dir, 'responses')
    os.makedirs(prompt_path, exist_ok=True)
    os.makedirs(response_path, exist_ok=True)
    
    # Requesting chat completion for each row
    df = i[2]   # Test data for ucoop or udef data
    for k in range(len(df)):
      row = df.iloc[k].to_dict()  # Creating a dictionary for each indv. row
      
      output = GPT_response(sys, str(row))  # GPT request output
      
      # Creating paths for prompts & GPT responses using window_numbers
      window_number = row['window_number']
      user_prmpt_path = os.path.join(prompt_path, f't{test[5:]}_{window_number}_user_prmpt.txt')
      output_path = os.path.join(response_path, f't{test[5:]}_{window_number}_response.txt')
      
      # Writing .txt files for prompts & GPT response
      write_file(user_prmpt_path, str(row))
      write_file(output_path, str(output))
  
  return print("Stage 2 Complete")


def get_testing_info(wtype, treat, stg, sum=True):  # Just making life easier for getting request file paths & data in sub-tests
  extension = f'{treat}_{wtype}'
  if sum == True:
    df_name = f'RAsum_{extension}.csv'
    test_df = pd.read_csv(f'test_data/{df_name}')
  else:
    df_name = f'{extension}.csv'
    test_df = pd.read_csv(f'test_data/{df_name}')
  
  if stg == 2:
    sys_path = f'code/prompts/{wtype}/{treat}/sys_2_{extension}.md'
    return sys_path, test_df
  else:
    sys_path = f'code/prompts/{wtype}/{treat}/sys_{str(stg)}_{extension}.md'
    user_path = f'code/prompts/{wtype}/{treat}/user_{str(stg)}_{extension}.md'
    return sys_path, user_path, df_name

In [4]:
##############
## Requests ##
##############

## Preliminaries
test = 'test_10'
window_type = 'ucoop'
treatment = 'noise'
summary_data = True
ucoop_test_data = pd.read_csv(f'test_data/RAsum_{treatment}_ucoop.csv')
udef_test_data = pd.read_csv(f'test_data/RAsum_{treatment}_udef.csv')
ucoop_test_data = ucoop_test_data[:20]
ucoop_test_data['window_number'] = ucoop_test_data['window_number'].astype(int)
udef_test_data = udef_test_data[:20]
udef_test_data['window_number'] = udef_test_data['window_number'].astype(int)

In [6]:
## Sub-test
sys, user, df_name = get_testing_info(window_type, treatment, 1, summary_data)
test_GPT(sys, user, df_name)

Sub-test 10055 complete


In [14]:
## Stage 1
stage_1_output(treatment, test)

Stage 1 Complete


In [15]:
## Stage 1 Refinement
stage_1r_output(treatment, test)

Stage 1 Refinement Complete


In [5]:
## Stage 2
stage_2_output(treatment, test, ucoop_test_data, udef_test_data)

Stage 2 Complete
